# Introduction

### Abstract

This is a solution for final problem from Autumn 2020 IBM Quantum Challenge. It uses the permutation approach, giving the circuit cost equal 6409. The main idea is to use an oracle with phase factor of $e^\frac{2 \pi i}{3}$ instead of $-1$ as in standard Grover's algorithm.

If you have any comments/questions, do not hesitate to contact me on qiskit Slack (adam klukowski) or at ak2028@cam.ac.uk.

### Problem statement

We are given 16 boards of dimensions 4x4 with six asteroids on each. Using a laser beam we can remove all asteroids on a chosen row/column. The task is to find the unique board that cannot be cleaned with at most three laser shots (it is allowed to use vertical and horizontal beams at the same time). We are asked to use Grover's algorithm with one iteration.

### Approach overview and motivation

Our circuit will use straightforward architecture:
 - Hadamard gates, to create equal superposition of address qubits
 - qRAM, to entangle address qubits with data qubits
 - oracle acting on data qubits
 - uncomputing qRAM
 - diffuser acting on address qubits

The oracle uses the following fact: a board is unsolvable iff it contains four asteroids with no two on the same row or column (a permutation matrix). Finding such arrangement can be thought of as a **dual problem** - each laser beam can eliminate at most one asteroid from such collection, so it is a "certificate" that we need at least four shots.

Naive way would be to iterate over all 24 permutations and apply CCCZ to each. This does not work - we may encounter a board with exactly two permutations, and two phase factors would cancel. Instead, we will use controlled rotation by $\frac{2\pi}{3}$; this way unsolvable boards will acquire phase factor of $\omega$ or $\omega^2$, where $\omega$ is primitive third root of unity.

The idea for qRAM is inspired by linear algebra over field of two elements. The circuit consists of a part that looks like a map from address states to basis of $\mathbb{Z}_2^{16}$, and another part that resembles a linear map.

### Impact of a different phase factor

It is natural to ask if full Grover's algorithm, beyond single iteration, is still possible with such oracle. It turns out that single oracle is not enough - one can obtain some aplification, but the amplitude will be bounded away from 1 (e.g. when searching among 1024 elements, the highest achievable probability of measuring the correct one is about 1%). However, we can overcome this by alternating between using phase factors $\omega = e^\frac{2 \pi i}{3}$ and $\omega^2 = e^\frac{-2 \pi i}{3}$. This comes at a cost: the rotation angle is smaller, so we need to apply a bit more iterations (still $O(\sqrt{size})$, but with a bigger constant).

In [1]:
# imports
from itertools import combinations
from math import pi
from qiskit import Aer, execute, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import RCCXGate
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

# original input
problem_set = \
    [[['0', '2'], ['1', '0'], ['1', '2'], ['1', '3'], ['2', '0'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '2'], ['2', '2'], ['3', '0'], ['3', '3']],
    [['0', '0'], ['1', '1'], ['1', '3'], ['2', '0'], ['3', '2'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '1'], ['1', '3'], ['3', '2'], ['3', '3']],
    [['0', '2'], ['1', '0'], ['1', '3'], ['2', '0'], ['3', '2'], ['3', '3']],
    [['1', '1'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '1'], ['3', '3']],
    [['0', '2'], ['0', '3'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '3']],
    [['0', '0'], ['0', '3'], ['1', '2'], ['2', '2'], ['2', '3'], ['3', '0']],
    [['0', '3'], ['1', '1'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '3'], ['2', '1'], ['2', '3'], ['3', '0']],
    [['0', '1'], ['0', '3'], ['1', '2'], ['1', '3'], ['2', '0'], ['3', '2']],
    [['0', '0'], ['1', '3'], ['2', '0'], ['2', '1'], ['2', '3'], ['3', '1']],
    [['0', '1'], ['0', '2'], ['1', '0'], ['1', '2'], ['2', '2'], ['2', '3']],
    [['0', '3'], ['1', '0'], ['1', '3'], ['2', '1'], ['2', '2'], ['3', '0']],
    [['0', '2'], ['0', '3'], ['1', '2'], ['2', '3'], ['3', '0'], ['3', '1']],
    [['0', '1'], ['1', '0'], ['1', '2'], ['2', '2'], ['3', '0'], ['3', '1']]]

# unroll and print cost
def print_cost(qc):
    ops = PassManager(Unroller(['u3', 'cx'])).run(qc).count_ops()
    print(ops, ', cost', ops['u3'] + 10 * ops['cx'])

# Preprocessing

We start with classical preprocessing. This function converts boards from list of coordinate pairs to list of asteroid indices. Indices range from 0 for ['0', '0'] to 15 for ['3', '3']. Repeated boards are problematic for this implementation of qRAM, so they are replaced with something solvable.

In [2]:
def preprocess(problem_set):
    
    # convert boards into lists of asteroid indices
    problem_set = [sorted([4 * int(i) + int(j) for i, j in board]) for board in problem_set]
    
    # remove repetitions
    repeated = 0
    for i in range(16):
        for j in range(i):
            if problem_set[i] == problem_set[j]:
                problem_set[i] = [repeated]
                repeated += 1
    
    return problem_set

# qRAM

### Architecture

qRAM consists of two parts:
 - Generic circuit calculating power of 2
 - Problemset-dependent circuit operating only on data qubits, expanding n-th qubit into the encoding of n-th board

The idea comes from linear algebra: first circuit can be treated as sending address states to canonical basis of $\mathbb{Z}_2^{16}$, and the second one as a linear map $\mathbb{Z}_2^{16} \rightarrow \mathbb{Z}_2^{16}$, sending canonical basis vectors to board encodings. However, in reality the second function is not linear, it only pretends to be.

### Power of 2

##### Function

qram_power_of_2 sends
$$ \lvert n \rangle \lvert 0 \rangle \mapsto \lvert n \rangle \lvert 2^n \rangle = \lvert n \rangle \lvert 0_0 \dots 0_{n-1} 1_n 0_{n+1} \dots 0_{16} \rangle $$
i.e. entangles state $\lvert n \rangle$ of address qubits with $n$-th data qubit.

##### Implementation

First we entangle address state $\lvert n \rangle$ with a subset of data qubits that contains the $n$-th. Equivalently, we send
$$ \lvert n \rangle \lvert 0 \rangle \mapsto \lvert n \rangle \lvert m \rangle $$
for some $m$ whose n-th binary digit is 1.

Then, we eliminate the undesired entanglements.

This is much clearer if we track the following: for $n$-th data qubit, what are the address states that can give 1 as its measurement?

After first part we have chosen data qubits entandled with
<table>
    <tr><th>data qubit</th><th>address states that have $\lvert 1 \rangle$ there</th>
    <tr><td>0</td><td>everything (always in state $\lvert 1 \rangle$)</td></tr>
    <tr><td>1</td><td>1, 5, 9, 13</td></tr>
    <tr><td>2</td><td>2, 6, 10, 14</td></tr>
    <tr><td>3</td><td>3, 7, 11, 15</td></tr>
    <tr><td>7</td><td>7, 15</td></tr>
    <tr><td>11</td><td>11, 15</td></tr>
    <tr><td>12</td><td>12, 13, 14, 15</td></tr>
    <tr><td>13</td><td>13, 15</td></tr>
    <tr><td>15</td><td>15</td></tr>
</table>
Then, we eliminate extra entaglements by taking symmetric differences. This is done by using CX gates: each CX(a, b) translates to xoring b-th subset with a-th subset. We ensure that we always xor a set with its subset with exactly half as many elements (e.g. CX(7, 3) changes set of 3 to $\{3, 7, 11, 15\} \triangle \{7, 15\} = \{3, 11\}$).

### Expansion

##### Function

qram_expand sends
$$\begin{align}
\lvert 2^n \rangle = \lvert 0_0 \dots 1_n \dots 0_{16} \rangle \mapsto \lvert 0_0 \dots 1_{a_1} \dots \dots \dots 1_{a_6} \dots 0_{16} \rangle \\
\text{where } a_1, \dots, a_6 \text{ are positions of asteroids on n-th board}
\end{align}$$

##### Implementation

This is arguably the least elegant bit of this solution.

The first idea is to iterate over each board, and add CX(data[n], data[$a_i$]) for each $a_i$ in n-th board - hoping that $n$-th qubit will flip all the asteroids from $n$-th board.

However, this will mix different boards and produce a mess. Therefore we will build this circuit inductively from layers:
 - Suppose we have the circuit for boards 0 to $i-1$
 - We compute the input state $\lvert \psi \rangle$ that will result in the $i$-th board. This is easy thanks to unitarity: we can prepare the state of $i$-th board, run the circuit _backwards_, and measure $\lvert \psi \rangle$
 - Prefix our circuit with a layer that sends $\lvert 2^i \rangle$ to $\lvert \psi \rangle$ and does not change $\lvert 2^j \rangle$ for $j<i$

##### Comments

This function uses _execute_, which competition submissions are not supposed to do. However, this computation is essentially classical (passing a string through some NOT and CNOT gates), so could be replaced with classical simulation.

Also, the algorithm used in this function has no optimization whatsoever, so it can probably be improved.

In [3]:
def qram_power_of_2(addr, data):
    
    pow2 = QuantumCircuit(addr, data)
    
    # Entangle n-th address state with a subset of data qubits
    for a1, a2 in combinations([0, 1, 2, 3], 2):
        pow2.append(RCCXGate(), [addr[a1], addr[a2], data[2 ** a1 + 2 ** a2]])
    for a1, a2 in [(3, 5), (3, 9), (3, 12), (5, 9), (6, 10)]:
        pow2.append(RCCXGate(), [data[a1], data[a2], data[a1 | a2]])
    for a, d, dd in [(0, 1, 3), (1, 2, 3), (2, 4, 6), (3, 8, 10)]:
        pow2.cx(addr[a], data[d])
        pow2.cx(data[dd], data[d])
    pow2.x(data[0])

    # Perform elimination
    pow2.cx(addr[0], data[0])
    pow2.cx(data[2], data[0])
    for s, d in [(8, 0), (7, [3, 5, 6]), (11, [9, 10]),
                 (14, 12), (5, 1), (6, 2), (12, 4), (9, [0, 8]),
                 (15, [7, 11, 13, 14]), (14, [6, 10]), (13, [5, 9, 12]), (11, 3),
                 (9, 1), (10, 2), (5, 4), (12, 8), (4, 0)]:
        pow2.cx(data[s], data[d])
    
    return pow2

def qram_expand(problem_set, data):
    lin = QuantumCircuit(data)
    aux_cr = ClassicalRegister(16)
    for i in range(16):

        # compute the state psi that will produce i-th board
        aux_circ = QuantumCircuit(data, aux_cr)
        for j in problem_set[i]: aux_circ.x(data[j])
        aux_circ += lin
        aux_circ.measure(data, aux_cr)
        aux_cnt = execute(aux_circ.reverse_bits(), Aer.get_backend('qasm_simulator'), shots = 1).result().get_counts()
        aux_state = list(aux_cnt.keys())[0]

        # add a layer sending 2^i to psi
        if aux_state[i] == '0':
            for j in range(i + 1, 16):
                if aux_state[j] == '1':
                    lin.cx(data[j], data[i])
                    break
            else:
                ones = [j for j in range(i) if aux_state[j] == '1']
                lin.append(RCCXGate(), [data[ones[0]], data[ones[1]], data[i]])
        for j in range(16):
            if aux_state[j] == '1' and j != i: lin.cx(data[i], data[j])

    return lin.reverse_ops()

print('qram_power_of_2:', end = ' ')
print_cost(qram_power_of_2(QuantumRegister(4), QuantumRegister(16)))
print('qram_expand:', end = ' ')
print_cost(qram_expand(preprocess(problem_set), QuantumRegister(16)))

qram_power_of_2: OrderedDict([('cx', 70), ('u3', 67)]) , cost 767
qram_expand: OrderedDict([('cx', 121), ('u3', 6)]) , cost 1216


# Oracle

##### Function

This adds a phase factor of $e^\frac{2 \pi i}{3}$ for every permutation on the board (or its conjugate when the argument "parity" takes odd value - check Introduction to see why).

##### Implementation

Naively, one could just add a triple-controlled rotation for each one of 24 permutations. We will essentially replicate this behaviour, but add some optimizations.

Observe that cells [0, 0] and [1, 1] appear in two permutations, so naive approach would include them twice. To avoid this, we can use four ancilla qubits (0-3), and apply to them a Toffoli gate with controls given by

                0 1 . .
                1 0 . .
                . . 2 3
                . . 3 2

Then we apply four controlled rotations on ancillas, each adding phase to permutations:
 - (0, 2), permutation [1, 2, 3, 4]
 - (0, 3), permutation [1, 2, 4, 3]
 - (1, 2), permutation [2, 1, 3, 4]
 - (1, 3), permutation [2, 1, 4, 3]

We need to apply this to 6 possible symmetric arrangements.

In [4]:
def oracle(data, ancilla, parity = 0):
    parity = 2 * parity - 1
    o = QuantumCircuit(data, ancilla)
    for col0, col1 in combinations([0, 1, 2, 3], 2):
        
        # rem_col are the columns for asteroids in two bottom rows
        rem_col = [i for i in range(4) if i!=col0 and i!=col1]
        
        # entangle pair of cells with ancillas
        for a, d1, d2 in [(0, col0, 4 + col1), (1, col1, 4 + col0), (2, 8 + rem_col[0], 12 + rem_col[1]), (3, 8 + rem_col[1], 12 + rem_col[0])]:
            o.append(RCCXGate(), [data[d1], data[d2], ancilla[a]])
            
        # apply phase factor
        for a1, a2 in [(0, 2), (0, 3), (1, 2), (1, 3)]:
            o.cu1(parity * 2 * pi / 3, ancilla[a1], ancilla[a2])
        
        # uncompute the entanglement
        for a, d1, d2 in [(0, col0, 4 + col1), (1, col1, 4 + col0), (2, 8 + rem_col[0], 12 + rem_col[1]), (3, 8 + rem_col[1], 12 + rem_col[0])]:
            o.append(RCCXGate(), [data[d1], data[d2], ancilla[a]])
    
    return o

print_cost(oracle(QuantumRegister(16), QuantumRegister(4)))

OrderedDict([('u3', 360), ('cx', 192)]) , cost 2280


<ipython-input-4-87de606164ce>:15: DeprecationWarning: The QuantumCircuit.cu1 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use the QuantumCircuit.cp method instead, which acts identically.
  o.cu1(parity * 2 * pi / 3, ancilla[a1], ancilla[a2])


# Putting it together

In [5]:
def diffusion(addr, ancilla):
    diff = QuantumCircuit(addr, ancilla)
    diff.h(addr)
    diff.x(addr)
    diff.h(addr[0])
    diff.mcx(addr[1:], addr[0], ancilla, 'v-chain')
    diff.h(addr[0])
    diff.x(addr)
    diff.h(addr)
    return diff
def week3_ans_func(problem_set):
    
    problem_set = preprocess(problem_set)
    
    addr, data, anc, cr = QuantumRegister(4), QuantumRegister(16), QuantumRegister(4), ClassicalRegister(4)
    
    ram = QuantumCircuit(addr, data)
    ram += qram_power_of_2(addr, data)
    ram += qram_expand(problem_set, data)
    
    qc = QuantumCircuit(addr, data, anc, cr)
    qc.h(addr)
    
    for i in range(1): # modify to use more iterations
        qc += ram
        qc += oracle(data, anc, i % 2)
        qc += ram.reverse_ops()
        qc += diffusion(addr, anc)
    
    qc.measure(addr, cr[::-1])
    
    return qc

print('diffuser:', end = ' ')
print_cost(diffusion(QuantumRegister(4), QuantumRegister(4)))
print('overall:', end = ' ')
print_cost(week3_ans_func(problem_set))

diffuser: OrderedDict([('u3', 39), ('cx', 12)]) , cost 159
overall: OrderedDict([('cx', 586), ('u3', 549), ('measure', 4)]) , cost 6409


# Result

In [4]:
# Submission code
from qc_grader import grade_ex3, prepare_ex3, submit_ex3

# Execute your circuit with following prepare_ex3() function.
# The prepare_ex3() function works like the execute() function with only QuantumCircuit as an argument.
job = prepare_ex3(week3_ans_func)

result = job.result()
counts = result.get_counts()
original_problem_set_counts = counts[0]

original_problem_set_counts
# The bit string with the highest number of observations is treated as the solution.

Running week3_ans_func...
Computing cost...
Starting experiments. Please wait...
You may monitor the job (id: 5fc3d36fe15c3b00192d39c8) status and proceed to grading when it successfully completes.


{'0000': 41,
 '0001': 39,
 '0010': 51,
 '0011': 35,
 '0100': 43,
 '0101': 359,
 '0110': 47,
 '0111': 46,
 '1000': 48,
 '1001': 50,
 '1010': 42,
 '1011': 36,
 '1100': 36,
 '1101': 43,
 '1110': 41,
 '1111': 43}

In [5]:
# Check your answer by executing following code.
# The quantum cost of the QuantumCircuit is obtained as the score. The lower the cost, the better.
grade_ex3(job)

Grading your answer. Please wait...

Congratulations 🎉! Your answer is correct.
Your score is 6409.
The lower your score the better!
Feel free to submit your answer.


In [6]:
# Submit your results by executing following code. You can submit as many times as you like during the period. 
submit_ex3(job)

Submitting your answer. Please wait...

Success 🎉! Your answer has been submitted.
Congratulations! You have rescued Dr. Ryoko from the quantum realm. The bright "quantum future" is ahead.
